In [ ]:
import torch
import os
import copy
import pandas as pd
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
# import pretraining_args as args
from transformers import BertConfig
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import BertModel
from transformers import BertTokenizer,BertConfig,BertForTokenClassification,BertModel
import time,datetime
from sklearn.metrics import precision_score,classification_report,f1_score,recall_score
import numpy as np
from torch.nn import CrossEntropyLoss, MSELoss
from transformers import AlbertConfig, AlbertModel,AlbertForTokenClassification
# from pytorch_pretrained_bert import BertTokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from crf import CRF
import time

In [ ]:
base = '/root/yy/data/ResumeNER'
base_path = '../berts/bert-base-transformers'
train_path = 'train.char.bmes'
dev_path = 'dev.char.bmes'
test_path = 'test.char.bmes'

In [ ]:
def load_data(base,train_path):
    full = os.path.join(base,train_path)
    with open(full,'r',encoding='utf-8')as f:
        data = f.readlines()
    tokens,labels = [],[]
    token,label = [],[]
    for line in data:
        line= line.strip().replace("\n",'')
        if len(line.split(' ')) == 2:
            token.append(line.split(' ')[0])
            label.append(line.split(' ')[1])
        else:
            tokens.append(token)
            labels.append(label)
            token,label = [],[]
    return tokens,labels

def trans2id(labels):
    tag_set = set()
    for line in labels:
        for label  in line:
            if label not in tag_set:
                tag_set.add(label)
    tag_set.add('[CLS]')
    tag_set.add('[SEP]')
    tag_set = list(tag_set)
    idx = [i for i in range(len(tag_set))]
    tag2id = dict(zip(tag_set,idx))
    id2tag = dict(zip(idx,tag_set))
    return tag2id,id2tag

def gen_features(tokens,labels,tokenizer,tag2id,max_len):
    input_ids,tags,masks,lengths = [],[],[],[]
    for i,(token,label) in enumerate(zip(tokens,labels)):
        lengths.append(len(token))
        if len(token) >= max_len - 2:
            token = token[0:max_len - 2]
            label = labels[i][0:max_len - 2]
        mask = [1] * len(token)
        
        token = '[CLS] ' + ' '.join(token) + ' [SEP]'
        tokenized_text = tokenizer.tokenize(token)
        input_id = tokenizer.convert_tokens_to_ids(tokenized_text)
        label = [tag2id['[CLS]']] + [tag2id[i] for i in label] + [tag2id['[SEP]']]
        mask = [0] + mask + [0]
        # padding
        if len(input_id) < max_len:
            input_id = input_id + [0] * (max_len - len(input_id))
            label = label + [tag2id['O']] * (max_len - len(label))
            mask = mask + [0] * (max_len - len(mask))
        
        assert len(input_id) == max_len
        assert len(label) == max_len
        assert len(mask) == max_len
         
        input_ids.append(input_id)
        tags.append(label)
        masks.append(mask)
    return input_ids,tags,masks,lengths

In [ ]:
max_len = 128
bs = 32
tokenizer = BertTokenizer.from_pretrained('../berts/bert-base-transformers')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_tokens,train_labels = load_data(base,train_path)
tag2id,id2tag = trans2id(train_labels)
train_ids,train_tags,train_masks,train_lengths = gen_features(train_tokens,train_labels,tokenizer,tag2id,max_len)

dev_tokens,dev_labels = load_data(base,dev_path)
dev_ids,dev_tags,dev_masks,dev_lengths = gen_features(dev_tokens,dev_labels,tokenizer,tag2id,max_len)

In [ ]:
config = BertConfig.from_json_file(os.path.join(base_path,'config.json'))
config.num_labels = len(tag2id)
bert = BertForTokenClassification.from_pretrained(os.path.join(base_path,'pytorch_model.bin'), config=config)

In [ ]:
train_ids = torch.tensor(train_ids)
train_tags = torch.tensor(train_tags)
train_masks = torch.tensor(train_masks)
# train_lengths = torch.tensor(train_lengths)

dev_ids = torch.tensor(dev_ids)
dev_tags = torch.tensor(dev_tags)
dev_masks = torch.tensor(dev_masks)
# dev_lengths = torch.tensor(dev_lengths)

In [ ]:
train_data = TensorDataset(train_ids, train_masks, train_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(dev_ids, dev_masks, dev_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [ ]:
optimizer = AdamW(bert.parameters(),
                  lr = 5e-5, # default is 5e-5
                  eps = 1e-8 # default is 1e-8
                )

In [ ]:
epochs = 10
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                           num_warmup_steps = 0,
                                           num_training_steps = total_steps)

In [ ]:
bert.to(device)

In [ ]:
def trans2label(id2tag,data,lengths):
    new = []
    for i,line in enumerate(data):
        tmp = [id2tag[word] for word in line]
        tmp = tmp[1:1 + lengths[i]]    
        new.append(tmp)
    return new

def get_entities(tags):
    start, end = -1, -1
    prev = 'O'
    entities = []
    n = len(tags)
    tags = [tag.split('-')[1] if '-' in tag else tag for tag in tags]
    for i, tag in enumerate(tags):
        if tag != 'O':
            if prev == 'O':
                start = i
                prev = tag
            elif tag == prev:
                end = i
                if i == n -1 :
                    entities.append((start, i))
            else:
                entities.append((start, i - 1))
                prev = tag
                start = i
                end = i
        else:
            if start >= 0 and end >= 0:
                entities.append((start, end))
                start = -1
                end = -1
                prev = 'O'
    return entities

def measure(preds,trues,lengths,id2tag):
    correct_num = 0
    predict_num = 0
    truth_num = 0
    pred = trans2label(id2tag,preds,lengths)
    true = trans2label(id2tag,trues,lengths)
    assert len(pred) == len(true)
    for p,t in zip(pred,true):
        pred_en = get_entities(p)
        true_en = get_entities(t)
        correct_num += len(set(pred_en) & set(true_en))
        predict_num += len(set(pred_en))
        truth_num += len(set(true_en))
    precision = correct_num / predict_num if predict_num else 0
    recall = correct_num / truth_num if truth_num else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0
    return f1, precision, recall

In [ ]:
max_grad_norm = 1.0

tra_loss,steps = 0.0,0
for i in range(epochs):
    bert.train()
    for step ,batch in enumerate(train_dataloader):
        input_ids,masks,labels= (i.to(device) for i in batch)
        outputs = bert(input_ids,attention_mask = masks,labels = labels)
        loss, scores = outputs[:2]
        loss.backward()
        
        tra_loss += loss
        steps += 1
        
        torch.nn.utils.clip_grad_norm_(parameters=bert.parameters(), max_norm=max_grad_norm)
        scheduler.step()
        optimizer.step()
        
        if step % 30 == 0:
            print("epoch :{},step :{} ,Train loss: {}".format(i,step,tra_loss/steps))
    
    print("Training Loss of epoch {}:{}".format(i,tra_loss / steps))
    
    bert.eval()
    dev_loss = 0.0
    predictions , true_labels = [], []
    
    for batch in valid_dataloader:
        input_ids,masks,labels = (i.to(device) for i in batch)
        with torch.no_grad():
            outputs = bert(input_ids,attention_mask = masks)
            #scores:(batch_size, sequence_length, config.num_labels) -> before softmax
            scores = outputs[0]
            
        scores = scores.detach().cpu().numpy()
        predictions.extend([list(p) for p in np.argmax(scores, axis=2)])
        true_labels.extend(labels.to('cpu').numpy().tolist())
#         lengths = lengths.detach().cpu().numpy().tolist()
#     dev_lengths = dev_lengths.detach().cpu().numpy()
    f1, precision, recall = measure(predictions,true_labels,dev_lengths,id2tag)
    print('epoch {} : Acc : {},Recall : {},F1 :{}'.format(i,precision,recall,f1))

In [ ]:
test_tokens,test_labels = load_data(base,test_path)
test_ids,test_tags,test_masks,test_lengths = gen_features(test_tokens,test_labels,tokenizer,tag2id,max_len)

In [ ]:
test_ids = torch.tensor(test_ids)
test_tags = torch.tensor(test_tags)
test_masks = torch.tensor(test_masks)

In [ ]:
test_data = TensorDataset(test_ids, test_masks, test_tags)
# test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, batch_size=bs)

In [ ]:
bert.eval()
test_pre,test_true = [],[]
for batch in test_dataloader:
    input_ids,masks,labels = (i.to(device) for i in batch)
    with torch.no_grad():
        outputs = bert(input_ids,attention_mask = masks)
        #scores:(batch_size, sequence_length, config.num_labels) -> before softmax
        scores = outputs[0]

    scores = scores.detach().cpu().numpy()
    test_pre.extend([list(p) for p in np.argmax(scores, axis=2)])
    test_true.extend(labels.to('cpu').numpy().tolist())

test_f1, test_precision, test_recall = measure(test_pre,test_true,test_lengths,id2tag)
# test_pred = trans2label(id2tag,test_pre,test_lengths)
# test_trues = trans2label(id2tag,test_true,test_lengths)
# measure_1(test_pre,test_true,test_lengths,id2tag,test_tokens)
print('Test Acc : {},Recall : {},F1 :{}'.format(test_precision,test_recall,test_f1))